The data used for this notebook can be downloaded from https://analyse.kmi.open.ac.uk/open_dataset. After unzipping the 'anonymisedData' folder, you can either place it in the directory of your code, or alter the directory references when reading the CSV files. No further preprocessing is required for the code of this notebook to work.

### Import libraries and read CSVs

In [ ]:
import pandas as pd
import numpy as np

df_vle = pd.read_csv('anonymisedData/vle.csv')
df_assessments = pd.read_csv('anonymisedData/assessments.csv')

df_student_vle = pd.read_csv('anonymisedData/studentVle.csv')
df_student_info = pd.read_csv('anonymisedData/studentInfo.csv')
df_student_assessment = pd.read_csv('anonymisedData/studentAssessment.csv')

df_student_vle.head()

### Create dictionaries with activity and module information

In [ ]:
id_site_dict = {}
for id_site, activity_type in zip(df_vle['id_site'], df_vle['activity_type']):
    if id_site in id_site_dict:
        continue
    else:
        id_site_dict[id_site] = activity_type
        
module_assignment_dates_dict = {}
for module, pres, date in zip(df_assessments['code_module'], df_assessments['code_presentation'], df_assessments['date']):
    if str(date) == 'nan':
        continue
        
    concat = module + '_' + pres
    if concat in module_assignment_dates_dict:
        module_assignment_dates_dict[concat]['before'].append(date - 1)
        module_assignment_dates_dict[concat]['on'].append(date)
        module_assignment_dates_dict[concat]['after'].append(date + 1)
    else:
        module_assignment_dates_dict[concat] = {'before': [date - 1], 'on': [date], 'after': [date + 1]}

### Create basic activity count features

In [ ]:
unique_ids = [f'{student}_{module}_{pres}' for student, module, pres in
                  zip(df_student_info['id_student'], df_student_info['code_module'], df_student_info['code_presentation'])]

features_df = pd.DataFrame({'id': unique_ids})
for (id_site, date_condition), group_df in df_student_vle.groupby(['id_site', df_student_vle['date'] >= 0]):
    group_df['id'] = [f'{student}_{module}_{pres}' for student, module, pres in
                  zip(group_df['id_student'], group_df['code_module'], group_df['code_presentation'])]
    
    feature_name = f'{"AC" if date_condition else "BC"}_{id_site_dict[id_site]}'

    feature_values = group_df.groupby('id')['sum_click'].sum().reset_index()
    
    temp_df = pd.DataFrame({'id': unique_ids})
    feature_values = pd.merge(temp_df, feature_values, on='id', how='left')
    feature_values.fillna(0, inplace=True)

    if feature_name in features_df.columns:
        features_df[feature_name] += feature_values['sum_click']
    else:
        features_df[feature_name] = feature_values['sum_click']

### Create dummy variables

In [ ]:
merge_df = pd.DataFrame({'id': unique_ids})

region_dummies = pd.get_dummies(df_student_info['region'])
imd_dummies = pd.get_dummies(df_student_info['imd_band'])
edu_dummies = pd.get_dummies(df_student_info['highest_education'])
age_dummies = pd.get_dummies(df_student_info['age_band'])

gender_binary = pd.Series(np.where(df_student_info['gender'] == 'F', 1, 0), name = 'gender')
disability_binary = pd.Series(np.where(df_student_info['disability'] == 'Y', 1, 0), name = 'disability')

merge_df = pd.concat([merge_df['id'], region_dummies.reset_index(drop=True), imd_dummies.reset_index(drop=True),
                      edu_dummies.reset_index(drop=True), age_dummies.reset_index(drop=True),
                      gender_binary.reset_index(drop=True), disability_binary.reset_index(drop=True),
                      df_student_info['num_of_prev_attempts'], df_student_info['studied_credits'],
                      df_student_info['final_result']], axis = 1)

features_df = pd.merge(features_df, merge_df, on='id', how='left')

### Create total click features

In [ ]:
df_student_vle['id'] = [f'{student}_{module}_{pres}' for student, module, pres in
                  zip(df_student_vle['id_student'], df_student_vle['code_module'], df_student_vle['code_presentation'])]

total_clicks = df_student_vle.groupby('id')['sum_click'].sum().reset_index()
total_clicks_bc = df_student_vle[df_student_vle['date'] < 0].groupby('id')['sum_click'].sum().reset_index()
total_clicks_ac = df_student_vle[df_student_vle['date'] >= 0].groupby('id')['sum_click'].sum().reset_index()

temp_df = pd.DataFrame({'id': unique_ids})

total_df = pd.merge(temp_df, total_clicks, on='id', how='left')
total_df.fillna(0, inplace=True)

total_bc_df = pd.merge(temp_df, total_clicks_bc, on='id', how='left')
total_bc_df.fillna(0, inplace=True)

total_ac_df = pd.merge(temp_df, total_clicks_ac, on='id', how='left')
total_ac_df.fillna(0, inplace=True)

features_df['total_clicks'] = total_df['sum_click']
features_df['BC_total_clicks'] = total_bc_df['sum_click']
features_df['AC_total_clicks'] = total_ac_df['sum_click']

### Create features of click count before, on, after assignment dates

In [ ]:
before_as_clicks = {}
on_as_clicks = {}
after_as_clicks = {}
for student_id, date, sum_click in zip(df_student_vle['id'],df_student_vle['date'],df_student_vle['sum_click']):
    pres_id = student_id[-9:]
    
    before_dates = module_assignment_dates_dict[pres_id]['before']
    on_dates = module_assignment_dates_dict[pres_id]['on']
    after_dates = module_assignment_dates_dict[pres_id]['after']
    
    if date in before_dates:
        if student_id in before_as_clicks:
            before_as_clicks[student_id] += sum_click
        else:
            before_as_clicks[student_id] = sum_click
    elif date in on_dates:
        if student_id in on_as_clicks:
            on_as_clicks[student_id] += sum_click
        else:
            on_as_clicks[student_id] = sum_click
    elif date in after_dates:
        if student_id in after_as_clicks:
            after_as_clicks[student_id] += sum_click
        else:
            after_as_clicks[student_id] = sum_click
    else:
        continue
        
temp_df = pd.DataFrame({'id': unique_ids})

before_clicks = pd.DataFrame(list(before_as_clicks.items()), columns=['id', 'sum_click'])
before_df = pd.merge(temp_df, before_clicks, on='id', how='left')
before_df.fillna(0, inplace=True)

on_clicks = pd.DataFrame(list(on_as_clicks.items()), columns=['id', 'sum_click'])
on_df = pd.merge(temp_df, on_clicks, on='id', how='left')
on_df.fillna(0, inplace=True)

after_clicks = pd.DataFrame(list(after_as_clicks.items()), columns=['id', 'sum_click'])
after_df = pd.merge(temp_df, after_clicks, on='id', how='left')
after_df.fillna(0, inplace=True)

features_df['Before_As_Clicks'] = before_df['sum_click']
features_df['On_As_Clicks'] = on_df['sum_click']
features_df['After_As_Clicks'] = after_df['sum_click']

### Create dataframes for each scenario and write to CSV

In [ ]:
def scenario_create_x_y(scenario, df):
    X = None
    y = None
    if scenario == 'pass-fail':
        df = df[df['final_result'] != 'Withdrawn']
        assert(len(df) == 22437)
        
        X = df.drop(['id', 'final_result'], axis = 1)
        y = pd.Series(df['final_result'].replace({'Pass': 0, 'Distinction': 0, 'Fail': 1}))
    elif scenario == 'fail-distinction':
        df = df[(df['final_result'] != 'Withdrawn') & (df['final_result'] != 'Pass')]
        assert(len(df) == 10076)
        
        X = df.drop(['id', 'final_result'], axis = 1)
        y = pd.Series(df['final_result'].replace({'Fail': 0, 'Distinction': 1}))
    elif scenario == 'pass-distinction':
        df = df[(df['final_result'] != 'Withdrawn') & (df['final_result'] != 'Fail')]
        assert(len(df) == 15385)
        
        X = df.drop(['id', 'final_result'], axis = 1)
        y = pd.Series(df['final_result'].replace({'Pass': 0, 'Distinction': 1}))
    elif scenario == 'pass-withdrawn':
        df = df[df['final_result'] != 'Fail']
        assert(len(df) == 25541)
        
        X = df.drop(['id', 'final_result'], axis = 1)
        y = pd.Series(df['final_result'].replace({'Pass': 0, 'Distinction': 0, 'Withdrawn': 1}))
    else:
        print('Invalid scenario. Please choose from: pass-fail, fail-distinction, distinction-pass, pass-withdrawn.')
        
    return X, y

In [ ]:
df_copy = features_df.copy()

X, y = scenario_create_x_y('pass-fail', df_copy)
X.to_csv('oulad_pass_fail_x.csv', index = False)
y.to_csv('oulad_pass_fail_y.csv', index = False)

X, y = scenario_create_x_y('fail-distinction', df_copy)
X.to_csv('oulad_fail_distinction_x.csv', index = False)
y.to_csv('oulad_fail_distinction_y.csv', index = False)

X, y = scenario_create_x_y('pass-distinction', df_copy)
X.to_csv('oulad_pass_distinction_x.csv', index = False)
y.to_csv('oulad_pass_distinction_y.csv', index = False)

X, y = scenario_create_x_y('pass-withdrawn', df_copy)
X.to_csv('oulad_pass_withdrawn_x.csv', index = False)
y.to_csv('oulad_pass_withdrawn_y.csv', index = False)